In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "8",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1",
        "spark.driver.memory": "2g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2933,application_1732639283265_2892,pyspark,idle,Link,Link,None,
2944,application_1732639283265_2903,pyspark,idle,Link,Link,None,
2947,application_1732639283265_2906,pyspark,idle,Link,Link,None,
2966,application_1732639283265_2924,pyspark,idle,Link,Link,None,
2971,application_1732639283265_2929,pyspark,idle,Link,Link,None,
2973,application_1732639283265_2931,pyspark,idle,Link,Link,None,


In [2]:
from sedona.spark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import year, to_date
from pyspark.sql.functions import substring, format_number
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import count, when, length
import time

from pyspark.sql.functions import count, when, col, min, avg, desc
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, round


# Create spark Session
spark = SparkSession.builder \
    .appName("query 5") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)
# Read the file from s3

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2975,application_1732639283265_2933,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
start_time = time.time()

crime19 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
    header=True
)

crime20 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    header=True
)

crime_df19 = crime19.select("LAT", "LON", "DR_NO")

crime_df20 = crime20.select("LAT", "LON", "DR_NO")
# Combine the two datasets
crime_df = crime_df19.union(crime_df20)

police_station = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv",
    header=True
)

police_station=police_station.select("DIVISION", "x", "y")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
crime_df = crime_df.withColumn("crime_point", ST_Point(col("LON"), col("LAT"))) #convert them to be fitted in sedona

police_station = police_station.withColumn("station_point", ST_Point(col("x"), col("y")))

crime_station_dist = crime_df.crossJoin(police_station) \
                .withColumn("DISTANCE", ST_DistanceSphere("crime_point", "station_point")/1000) #calc dist + conv to km

crime_station_dist = crime_station_dist.select("DIVISION", "DISTANCE", "DR_NO")
# crime_station_dist.show(30)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
window_spec = Window.partitionBy("DR_NO").orderBy("DISTANCE")
crime_station_dist = crime_station_dist.withColumn("rank", row_number().over(window_spec))

crime_station_dist = crime_station_dist.filter(col("rank") == 1).drop("rank")

average_crime_station_dist = crime_station_dist.groupBy("DIVISION").agg(avg("DISTANCE").alias("average_distance"))

average_crime_station_dist = average_crime_station_dist.withColumn(
    "average_distance",
    format_number(col("average_distance"), 5)
)

no_crime_station_dist =  crime_station_dist.groupBy("DIVISION").agg(count("DIVISION").alias("#"))
#no_crime_station_dist.show(10)
final_df = average_crime_station_dist.join(no_crime_station_dist, on="DIVISION",how="inner").orderBy(desc("#"))
final_df.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------+------+
|   DIVISION|average_distance|     #|
+-----------+----------------+------+
|  HOLLYWOOD|         2.07626|224340|
|   VAN NUYS|         2.95337|210134|
|  SOUTHWEST|         2.19140|188901|
|   WILSHIRE|         2.59267|185996|
|77TH STREET|         1.71654|171827|
+-----------+----------------+------+
only showing top 5 rows

### end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

In [7]:
# 2 executors x 4 cores/8GB memory: 26.77s
# 4 executors x 2 cores/4GB memory: 30.79s
# 8 executors x 1 cores/2GB memory: 40.81s

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…